<a href="https://colab.research.google.com/github/AmarnathMishra777/Network-File-Sharing-Server-Client/blob/main/Network_File_Sharing_Server_%26_Client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import socket
import os
import threading
import time

HOST = "127.0.0.1"
PORT = 5055            # use a fresh port each run
SHARED_DIR = "shared"

if not os.path.exists(SHARED_DIR):
    os.makedirs(SHARED_DIR)

# ---------------- SERVER ----------------
def server_program():
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server.bind((HOST, PORT))
    server.listen(1)
    print(f"✅ Server started on {HOST}:{PORT}, waiting for client...")

    conn, addr = server.accept()
    print(f"🔗 Client connected from {addr}")

    while True:
        try:
            command = conn.recv(1024).decode()
            if not command:
                break
        except:
            break

        # ---------- List files ----------
        if command == "LIST":
            files = os.listdir(SHARED_DIR)
            if not files:
                conn.send(b"No files available.")
            else:
                conn.send("\n".join(files).encode())

        # ---------- Download file ----------
        elif command.startswith("DOWNLOAD"):
            filename = command.split(" ", 1)[1]
            path = os.path.join(SHARED_DIR, filename)
            if os.path.exists(path):
                conn.send(b"EXISTS")
                with open(path, "rb") as f:
                    while True:
                        data = f.read(1024)
                        if not data:
                            break
                        conn.sendall(data)
                time.sleep(0.2)
                conn.sendall(b"<END>")
                print(f"📤 Sent file: {filename}")
            else:
                conn.send(b"NOFILE")

        # ---------- Upload file ----------
        elif command.startswith("UPLOAD"):
            filename = command.split(" ", 1)[1]
            path = os.path.join(SHARED_DIR, filename)
            conn.send(b"READY")
            with open(path, "wb") as f:
                while True:
                    data = conn.recv(1024)
                    if data.endswith(b"<END>"):
                        f.write(data[:-5])
                        break
                    f.write(data)
            print(f"📥 Received file: {filename}")

        # ---------- Exit ----------
        elif command == "EXIT":
            print("❌ Client disconnected.")
            break

    conn.close()
    server.close()

# ---------------- CLIENT ----------------
def client_program():
    time.sleep(1.5)  # allow server to start
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.settimeout(10)
    client.connect((HOST, PORT))
    print("✅ Connected to server.")

    # 1️⃣ List files
    client.send(b"LIST")
    try:
        data = client.recv(4096).decode()
    except socket.timeout:
        print("⚠️ Timeout receiving file list.")
        client.close()
        return
    print("\n📂 Files on server:\n", data)

    # 2️⃣ Upload a file
    filename = "demo.txt"
    with open(filename, "w") as f:
        f.write("Hello from Colab client! 🚀")

    client.send(f"UPLOAD {filename}".encode())
    if client.recv(1024) == b"READY":
        with open(filename, "rb") as f:
            while (data := f.read(1024)):
                client.send(data)
        client.send(b"<END>")
    print(f"✅ Uploaded {filename}")

    # 3️⃣ Wait a bit, then download it back
    time.sleep(1)
    client.send(f"DOWNLOAD {filename}".encode())
    try:
        resp = client.recv(1024)
    except socket.timeout:
        print("⚠️ No response for download.")
        client.close()
        return

    if resp == b"EXISTS":
        with open("downloaded_" + filename, "wb") as f:
            while True:
                try:
                    data = client.recv(1024)
                except socket.timeout:
                    print("⚠️ Download timed out.")
                    break
                if data.endswith(b"<END>"):
                    f.write(data[:-5])
                    break
                f.write(data)
        print(f"✅ Downloaded file: downloaded_{filename}")
    else:
        print("❌ File not found on server.")

    # 4️⃣ Exit
    client.send(b"EXIT")
    client.close()
    print("🔒 Connection closed.")

# Run both threads
threading.Thread(target=server_program, daemon=True).start()
client_program()


✅ Server started on 127.0.0.1:5055, waiting for client...
✅ Connected to server.🔗 Client connected from ('127.0.0.1', 47982)


📂 Files on server:
 demo.txt
✅ Uploaded demo.txt
📥 Received file: demo.txt
📤 Sent file: demo.txt
✅ Downloaded file: downloaded_demo.txt
🔒 Connection closed.
❌ Client disconnected.
